This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [4]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '2zmAGAcomRuVxHMzssWh'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import collections
import requests

In [6]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
data = requests.get('https://www.quandl.com/api/v3/datasets/WIKI/FB.json?start_date=2017-07-07&end_date=2017-07-07&api_key={}'.format(API_KEY))
dic = data.json()
dic


{'dataset': {'id': 9775687,
  'dataset_code': 'FB',
  'database_code': 'WIKI',
  'name': 'Facebook Inc. (FB) Prices, Dividends, Splits and Trading Volume',
  'description': "End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Facebook, Inc. (FB). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (www.crsp.com/products/documentation/crsp-calculations)\n\nThis data is in the public domain. You may copy, distribute, disseminate or include the data in other products for commercial and/or noncommercial purposes.\n\nThis data is part of Quandl's Wiki initiative to get financial data permanently into the public domain. Quandl relies on users like you to flag errors and provide data where data is wrong or missing. Get involved: connect@quandl.com\n",
  'refreshed_at': '2018-03-27T21:46:11.036Z',
  'newest_available_date': '2018-03-27',
  'old

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
# Collect data and convert into python dictionary
data = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key={}'.format(API_KEY))
dic = data.json()
dataset = dic['dataset']


In [22]:
# Calculate what the highest and lowest opening prices were for the stock in this period.

open_ind = dataset['column_names'].index('Open')
hi, lo = 0, float('inf')
for d in dataset['data']:
    if isinstance(d[open_ind], float):
        if d[open_ind] > hi:
            hi = d[open_ind]
        if d[open_ind] < lo:
            lo = d[open_ind]
print('highest opening: {} \nlowest opening: {}'.format(hi, lo))

highest opening: 53.11 
lowest opening: 34.0


In [23]:
# What was the largest change in any one day (based on High and Low price)?

hi_ind, low_ind = dataset['column_names'].index('High'), dataset['column_names'].index('Low')
diff = 0
for d in dataset['data']:
    dif = d[hi_ind] - d[low_ind]
    if dif > diff:
        diff = dif
print('largest change is: {}'.format(diff))

largest change is: 2.8100000000000023


In [30]:
# What was the largest change between any two days (based on Closing Price)?
# First Interpretation: assuming two consecutive days
clo_ind = dataset['column_names'].index('Close')
change = 0
for i in range(1, len(dataset['data'])):
    x = dataset['data'][i][clo_ind] - dataset['data'][i-1][clo_ind]
    if abs(x) > abs(change):
        change = x
print('largest change between any two consecutive days: ', change)

# Second Interpretaiton: assuming any two days in whole year
highest_close = 0
lowest_close = float('inf')
for i in range(1, len(dataset['data'])):
    cur_close = dataset['data'][i][clo_ind]
    if cur_close > highest_close:
        highest_close = cur_close
    elif cur_close < lowest_close:
        lowest_close = cur_close
change = highest_close-lowest_close
print('largest change between any two days in whole year: ', change)

# closing_prices = [x[4] for x in dataset['data']]
# valid_closing_values = list(filter(lambda x:isinstance(x, float), closing_prices))
# highest_closing = max(valid_closing_values)
# lowest_closing = min(valid_closing_values)
# largest_change = highest_closing - lowest_closing
# print(f"validating my answer: {largest_change}")

largest change between any two consecutive days:  2.559999999999995
largest change between any two days in whole year:  19.03


In [25]:
# What was the average daily trading volume during this year?

tv_ind = dataset['column_names'].index('Traded Volume')
arr = []
for d in dataset['data']:
    arr.append(d[tv_ind])
print('average daily trading volumn is: ', sum(arr)/len(arr))

average daily trading volumn is:  89124.33725490196


In [26]:
import heapq
# What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

tv_ind = dataset['column_names'].index('Traded Volume')
arr = []
for d in dataset['data']:
    arr.append(d[tv_ind])
heapq.heapify(arr)
ind = len(dataset['data'])//2+1 if len(dataset['data'])%2 else len(dataset['data'])//2
for i in range(ind):
    median = heapq.heappop(arr)
print('median trading volumn is: ', median)

median trading volumn is:  76286.0
